In [2]:
import torch
import json

In [3]:
goldpath_file = "/opt/data/scienceworld-goldpaths/goldsequences-0.json"
with open(goldpath_file) as f:
    data = json.load(f)
json_data = data['0']

gold_sequence = json_data['goldActionSequences'][0]['path']
goal = json_data['goldActionSequences'][0]['taskDescription'].split('.')[0]
variation_idx = json_data['goldActionSequences'][0]['variationIdx']
print(f"Goal: {goal} - variation {variation_idx}")

last_reward = 0
use_cls = True
observation = ""
all_trajectories = []
for i, trajectory in enumerate(gold_sequence):
    look_around = trajectory['freelook']
    inventory = trajectory['inventory']
    full_state = look_around + inventory + trajectory['observation']
    print(f"Step {i}: {trajectory['action']} - num tokens {len(full_state)}")

Goal: Your task is to boil water - variation 0
Step 0: look around - num tokens 738
Step 1: open door to kitchen - num tokens 414
Step 2: go to kitchen - num tokens 413
Step 3: look around - num tokens 1884
Step 4: pick up thermometer - num tokens 1004
Step 5: open cupboard - num tokens 987
Step 6: pick up metal pot - num tokens 1131
Step 7: look around - num tokens 2038
Step 8: move metal pot to sink - num tokens 1126
Step 9: activate sink - num tokens 1108
Step 10: deactivate sink - num tokens 1126
Step 11: pick up metal pot - num tokens 1139
Step 12: focus on substance in metal pot - num tokens 1131
Step 13: pour metal pot into metal pot - num tokens 1145
Step 14: pick up metal pot - num tokens 1148
Step 15: move metal pot to stove - num tokens 1144
Step 16: activate stove - num tokens 1126
Step 17: examine substance in metal pot - num tokens 1122
Step 18: use thermometer in inventory on substance in metal pot - num tokens 1158
Step 19: examine substance in metal pot - num tokens 11

In [22]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

# Import our models. The package will take care of downloading the models automatically
model_name = "princeton-nlp/sup-simcse-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Tokenize input texts
texts = [
    "focus on substance in metal pot",
    "focus on metal pot",
    "pour metal pot into metal pot"
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
print(embeddings.size())
# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])

print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))


torch.Size([3, 1024])
Cosine similarity between "focus on substance in metal pot" and "focus on metal pot" is: 0.934
Cosine similarity between "focus on substance in metal pot" and "pour metal pot into metal pot" is: 0.763


In [10]:
trajectory['inventory']

'In your inventory, you see:\n\ta thermometer, currently reading a temperature of 10 degrees celsius\n\tan orange\n'

In [18]:
inv_fragments = trajectory['inventory'].split("\n")
inv_fragments = [i.replace("\t", "") for i in inv_fragments if len(i) > 0]
inv_preamble = inv_fragments[0]
inv_fragments = [f"{inv_preamble} {i}" for i in inv_fragments[1:]]
inv_fragments

['In your inventory, you see: a thermometer, currently reading a temperature of 10 degrees celsius',
 'In your inventory, you see: an orange']

In [40]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# 1. Specify preffered dimensions
dimensions = 512

# 2. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)

# For retrieval you need to pass this prompt.
query = "focus on substance in metal pot"

docs = [
    query,
    "focus on metal pot",
    "pour metal pot into metal pot"
]

# 2. Encode
embeddings = model.encode(docs)

# Optional: Quantize the embeddings
binary_embeddings = quantize_embeddings(embeddings, precision="uint8")

similarities = cos_sim(binary_embeddings[0], binary_embeddings[1:])
print('similarities:', similarities)


/home/ichida/miniconda3/envs/default_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Computing uint8 quantization buckets based on 3 embeddings. uint8 quantization is more stable with `ranges` calculated from more embeddings or a `calibration_embeddings` that can be used to calculate the buckets.


RuntimeError: linalg.vector_norm: Expected a floating point or complex tensor as input. Got Byte

In [39]:
binary_embeddings

array([[  0,  29, 255, ..., 147, 241, 255],
       [ 64,   0,   0, ..., 255, 255,   0],
       [255, 255, 183, ...,   0,   0, 182]], dtype=uint8)